# Dynamic Programming

- **Created by Andrés Segura Tinoco**
- **Created on Jan 26, 2020**

In [1]:
# Load the Python libraries
import timeit

## 1. Binomial Coefficient

In [2]:
# Example values
n = 20
k = 10

### Simple approach

In [3]:
def C(n, k):
    if k == 0 or k == n:
        return 1
    return C(n - 1, k - 1) + C(n - 1, k)

In [4]:
start_time = timeit.default_timer()
print(C(n, k))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

184756
>> elapsed time 124.23419999999999 ms


### With Dynamic Programming

In [5]:
def C2(n, k):
    c = 0
    v = [1] * (k + 1)
    
    for i in range(n + 1):
        for j in range(k, 0, -1):
            if j < i:
                v[j] = v[j - 1] + v[j]
    
    return v[k]

In [6]:
start_time = timeit.default_timer()
print(C2(n, k))
print('>> elapsed time', (timeit.default_timer() - start_time) * 1000, 'ms')

184756
>> elapsed time 1.0985000000000023 ms


With computational complexity of $ \Theta(nk) $ and a time complexity of $ \Theta(k) $.

---
<a href="https://ansegura7.github.io/Algorithms/">« Home</a>